# Extract features + FPDAA labels

This notebook pulls FPDAA labels from SQL and extracts model input features from rerun output JSON files.


In [15]:
from __future__ import annotations

import json
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine 
import urllib

server = '192.168.1.15'
username = 'Junchen'
password = '9DFXjhhlR3k5'
database = 'LF_LMSMaster'

params = urllib.parse.quote_plus(
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server},1433;DATABASE={database};UID={username};PWD={password};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)
engine = create_engine('mssql+pyodbc:///?odbc_connect=%s' % params)


cnxn = engine.connect()
query1 = """ 
USE LF_LMSMASTER
DROP TABLE IF EXISTS #t1;
SELECT
    A.Application_ID, A.ApplicationSteps, A.LPCampaign, D.DenialCode, D.DenialDescription,A.PortfolioID, A.CustomerID, A.CustomerSSN, A.ApplicationDate, A.Score, CustEmail,
    CASE WHEN ApplicationSteps LIKE '%S%' THEN 1 ELSE 0 END AS NewlyScored,
    CASE WHEN A.ApplicationStatus IN ('A','P') THEN 1 ELSE 0 END AS Accepted,
    CASE WHEN A.ApplicationStatus IN ('A','P') THEN A.LeadPurchasePrice ELSE 0 END AS LeadPurchasePrice,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') AND A.ApplicationStatus='J' THEN 1 ELSE 0 END AS PartialPreApproved,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') THEN 1 ELSE 0 END AS Originated,
    L.LoanID,
    DATEDIFF(YEAR, VW.DOB, A.ApplicationDate) AS Age,
    CASE WHEN VW.Frequency IN ('B','S') THEN 'B' ELSE VW.Frequency END AS Frequency,
    CASE WHEN L.LoanStatus NOT IN ('V','W','G','K') THEN L.OriginatedAmount ELSE NULL END AS OriginatedAmount,
    OriginationDate,
    LoanStatus,
    CASE WHEN ((L.RenewalLoanId <> '0') OR (A.LPCampaign = 'RENEW')) THEN 'RENEWAL'
         WHEN ApplicationSteps NOT LIKE '%R%' THEN 'NEW'
         ELSE 'REPEAT' END AS CustType,
    RenewalLoanId
INTO #t1
FROM Application A
LEFT JOIN Loans L
  ON A.Application_ID = L.ApplicationID AND A.PortFolioID = L.PortFolioID
LEFT JOIN LF_LMS_Logs..VW_ApplicationDump VW
  ON A.APPGUID = VW.APPGUID
LEFT JOIN LF_LMSMASTER..DenialCode D
  ON A.DenialCode = D.DenialCode

where ((VW.CustEmail is null) or  (VW.CustEmail not in ( 'josh@concordadvice.com', 'tiffany.speedyservicing@gmail.com','bobby@speedyservicing.com',
'and.kom@example.com', 'and.tor020@example.com', 'and.tor050@example.com', 'and.torrc4@example.com',
'test@dot818.com', 'test@example.com', 'test@gmail.com', 'test@loanscanada.ca', 'test@test.com', 'test2@test.com',
'testerqad@gmail.com', 'testik@test.com', 'testteam@dmaassociatescorp.com', 'tiffany.speedyservicing@gmail.com')))


UPDATE #t1 -- update the renewal loanid from 0 to correct previous loanIDs
SET #t1.RenewalLoanId = B.RenewalLoanId
from #t1 A inner join (select * from
(select A.LoanID, L.LoanID as RenewalLoanId, row_number() over (partition by A.LoanID order by datediff(day, L.OriginationDate, A.OriginationDate) desc) as RN from #t1 A
inner join Application A2 on A.CustomerID = A2.CustomerID and A2.ApplicationDate < A.ApplicationDate
inner join Loans L on A2.Application_ID = L.ApplicationID and A2.PortFolioID = L.PortFolioID and L.OriginationDate < A.OriginationDate and L.LoanStatus not in ('W','V','G','K')
where A.CustType = 'RENEWAL' and A.RenewalLoanId = '0') K where RN=1) B on A.LoanID = B.LoanID

------------ narrow to only originated ------------------
DROP TABLE IF EXISTS #t2;

SELECT
    *
INTO #t2
FROM #t1
WHERE Originated = 1; 

DROP TABLE IF EXISTS #t2_perf;
SELECT
    A.*,

    /* First-installment (maturity) flag */
    CASE
        WHEN A.LoanStatus NOT IN ('V','W','G','K')
             AND NOT (
                 inst.iPaymentMode = 144
                 AND inst.Pending = 1
                 AND inst.DueDate >= CAST(GETDATE() AS date)
             )
        THEN 1 ELSE 0
    END AS is_loan_first_install,

    /* FPDAA (NULL -> 0). This is a per-row flag for modeling / general use. */
    COALESCE(pay.FPDAA_raw, 0) AS FPDAA,

    /* Metrics-safe FPDAA: only evaluate when mature; otherwise NULL */
    CASE
        WHEN A.LoanStatus NOT IN ('V','W','G','K')
             AND NOT (
                 inst.iPaymentMode = 144
                 AND inst.Pending = 1
                 AND inst.DueDate >= CAST(GETDATE() AS date)
             )
        THEN COALESCE(pay.FPDAA_raw, 0)
        ELSE NULL
    END AS FPDAA_matured

INTO #t2_perf
FROM #t2 A

/* Pull first installment attributes (for maturity logic) */
OUTER APPLY (
    SELECT TOP 1
        I.DueDate,
        I.iPaymentMode,
        CASE WHEN I.Status = 684 THEN 1 ELSE 0 END AS Pending
    FROM LF_LMSMaster..Installments I
    WHERE I.LoanID = A.LoanID
      AND I.InstallmentNumber = 1
    ORDER BY I.DueDate DESC, I.InstallmentID DESC
) inst

/* Compute FPDAA_raw using your “1 - MAX(paid_off)” logic, per LoanID */
OUTER APPLY (
    SELECT
        1 - MAX(CASE WHEN P.PaymentStatus = 'D' THEN 1 ELSE 0 END) AS FPDAA_raw
    FROM LF_LMSMaster..Payment P
    WHERE P.LoanID = A.LoanID
      AND P.PaymentMode IN ('A','D','B')
      AND P.PaymentType NOT IN ('3','~','Q')
      AND P.InstallmentNumber = 1
      AND P.AttemptNo >= 1
      AND P.PaymentDate <= GETDATE()
      AND P.PaymentStatus IN ('D','R','S','B')
) pay;
DROP TABLE IF EXISTS #t2_ibv;
SELECT ibv.IBVStatusID, IBV.DateCreated, A1.*
INTO #t2_ibv
FROM #t2_perf AS A1 LEFT JOIN [LF_BankData].[dbo].[IBVStatus] AS ibv ON A1.CustomerSSN = ibv.AccountNumber
--ibv.IBVStatusID, ibv.DateCreated

drop table if exists #t2_ibv_dedup
SELECT *
into #t2_ibv_dedup
FROM (
 SELECT *, ROW_NUMBER() OVER (PARTITION BY LoanID ORDER BY DATEDIFF(day, ApplicationDate, DateCreated)) AS row_number
 FROM #t2_ibv
) AS t
WHERE t.row_number = 1

DROP TABLE IF EXISTS #ibv_to_apps;
SELECT rz.IBVToken, t1.*
INTO #ibv_to_apps
FROM #t2_ibv_dedup AS t1
JOIN (
    SELECT IBVToken
    FROM BankuityPostOnboarding.dbo.SpeedyAnalysis
    WHERE ExperimentName = 'loonie_rerun_V3'
) rz
  ON t1.IBVStatusID = rz.IBVToken;

"""
query2 = """
SELECT *
FROM #ibv_to_apps

"""

with engine.begin() as cnxn:  # same connection + transaction
    cnxn.exec_driver_sql("SET NOCOUNT ON; USE LF_LMSMASTER;")
    cnxn.exec_driver_sql(query1)               # multi-statement OK here
    df_perf_orig = pd.read_sql(query2, cnxn)        # sees temp tables
# df_perf = pd.read_csv('perf_accept_orig_formodeling_dependentmeasure.csv')
df_perf_orig.rename(columns = {'Application_ID':'AppID'}, inplace=True)

print(df_perf_orig.shape)



(5704, 31)


In [16]:
df_perf_orig.head(10)

,IBVToken,IBVStatusID,DateCreated,AppID,ApplicationSteps,LPCampaign,DenialCode,DenialDescription,PortfolioID,CustomerID,...,Frequency,OriginatedAmount,OriginationDate,LoanStatus,CustType,RenewalLoanId,is_loan_first_install,FPDAA,FPDAA_matured,row_number
0,10085,10085,2024-03-28 17:08:11.407,57506,RU,RETURN,0,Accepts,1,9089,...,B,1000.0,2025-03-17 13:00:26.073,D,REPEAT,0,1,0,0.0,1
1,36722,36722,2025-03-19 09:22:44.580,58593,U,RETURN,0,Accepts,1,34858,...,B,300.0,2025-03-21 12:23:03.207,D,NEW,0,1,0,0.0,1
2,34868,34868,2025-03-06 09:35:40.963,54948,US,WEB,0,Accepts,1,33181,...,B,500.0,2025-03-06 10:01:30.220,D,NEW,0,1,0,0.0,1
3,39901,39901,2025-04-09 13:39:04.670,63200,UUS,WEB,0,Accepts,1,37439,...,B,500.0,2025-04-09 14:18:26.667,D,NEW,0,1,0,0.0,1
4,54019,54019,2025-06-18 16:48:54.397,87341,UUS,WEB,0,Accepts,1,49494,...,M,500.0,2025-06-20 12:20:05.423,D,NEW,0,1,0,0.0,1
5,54424,54424,2025-06-22 12:23:32.967,88046,UUS,LLC012BA1PES,0,Accepts,1,49821,...,B,300.0,2025-06-23 16:36:12.637,R,NEW,0,1,0,0.0,1
6,63398,63398,2025-08-11 04:38:08.083,102807,UUS,LLC012BA1PES,0,Accepts,1,56557,...,W,600.0,2025-08-11 15:55:05.287,D,NEW,0,1,0,0.0,1
7,63580,63580,2025-08-11 17:16:20.180,103087,UUS,LLC012BA1PES,0,Accepts,1,56701,...,B,350.0,2025-08-11 17:55:18.773,D,NEW,0,1,0,0.0,1
8,63662,63662,2025-08-12 11:42:20.217,103224,UUS,LLC012BA1PES,0,Accepts,1,56764,...,B,350.0,2025-08-12 16:53:55.320,R,NEW,0,1,0,0.0,1
9,42924,42924,2025-04-26 21:42:13.433,103233,UUS,LLC012BA1PES,0,Accepts,1,40035,...,W,350.0,2025-08-13 15:15:51.240,D,NEW,0,1,0,0.0,1


In [ ]:
# --- Feature extraction from rerun output JSONs ---
input_dir = "/Users/starsrain/nov2025_concord/loonie_bankuity_rerun/rerun_output_JSONs_V3"
feature_level = "accountLevel"  # or "customerLevel"


def _get_score_feature_rows(data: dict, level: str) -> list[dict] | None:
    scores = data.get("customerInfo", {}).get("scores", {})
    feature_block = scores.get("features", {})
    rows = feature_block.get(level)
    if isinstance(rows, list) and rows:
        return rows
    return None


def _get_account_feature_rows(data: dict) -> list[dict] | None:
    accounts = data.get("accounts")
    if not isinstance(accounts, list) or not accounts:
        return None
    rows = []
    for account in accounts:
        features = account.get("features")
        if isinstance(features, dict):
            row = dict(features)
            row["accountGuid"] = account.get("accountGuid")
            rows.append(row)
    return rows or None


def extract_features_from_output(data: dict, level: str) -> list[dict]:
    feature_rows = _get_score_feature_rows(data, level) or _get_account_feature_rows(data) or []

    # currentBalance is a model feature but not inside the feature dict
    account_balance = {
        acc.get("accountGuid"): acc.get("currentBalance")
        for acc in data.get("accounts", [])
        if isinstance(acc, dict)
    }
    ibv_status_id = data.get("ibv_status_id")
    for row in feature_rows:
        account_guid = row.get("accountGuid")
        if account_guid in account_balance and "currentBalance" not in row:
            row["currentBalance"] = account_balance[account_guid]
        if ibv_status_id is not None:
            row["IBVStatusID"] = ibv_status_id
    return feature_rows


rows = []
row_meta = []
for json_path in Path(input_dir).glob("*.json"):
    with json_path.open() as f:
        data = json.load(f)
    extracted = extract_features_from_output(data, feature_level)
    rows.extend(extracted)
    row_meta.append({"file": json_path.name, "row_count": len(extracted)})

features_df = pd.DataFrame(rows)
row_counts_df = pd.DataFrame(row_meta)

features_df.head()



                            accountGuid  activeMonthlyIncome  active_count_0  \
0  57d0c888-005a-46b8-b0b7-a3209581f778              1287.00               2   
1  ff39bfe3-5e68-4afa-bda6-c21d80ce5185              1104.40               4   
2  dbec71d6-ca33-49ab-b25c-2bf5f8f108ed              9678.49               1   
3  ceb777fe-2bfd-4e3b-b0fd-ccf4cc7ab6c9              5287.21               1   
4  931d1df2-04c6-41a8-a2f2-190598c10c69                 0.00               1   

   active_count_1  active_count_2  active_count_3  active_monthly_0  \
0               2               0               1          11154.54   
1               0               0               1           6289.35   
2               0               1               1           1496.21   
3               0               0               1           4984.64   
4               0               0               0             47.73   

   active_monthly_1  active_monthly_2  active_monthly_3  ...  \
0           4221.98         

In [14]:
print(features_df.shape)

(11843, 104)


In [17]:
# --- Analysis: row counts per JSON + inspect multi-account file ---
print("json files:", len(row_counts_df))
print("feature rows:", len(features_df))
print("files with >1 account:", (row_counts_df["row_count"] > 1).sum())
row_counts_df["row_count"].value_counts().sort_index().head(10)

# Inspect one JSON file with multiple accounts
multi_file = row_counts_df.loc[row_counts_df["row_count"] > 1, "file"].head(1)
if not multi_file.empty:
    multi_path = Path(input_dir) / multi_file.iloc[0]
    with multi_path.open() as f:
        multi_data = json.load(f)
    multi_rows = extract_features_from_output(multi_data, feature_level)
    multi_df = pd.DataFrame(multi_rows)
    print("multi-account file:", multi_file.iloc[0])
    print("accounts in file:", len(multi_df))
    display(multi_df.head())
else:
    print("No multi-account files found.")


json files: 11474
feature rows: 11843
files with >1 account: 278
multi-account file: ModelResponse_86334.json
accounts in file: 3


,accountGuid,activeMonthlyIncome,active_count_0,active_count_1,active_count_2,active_count_3,active_monthly_0,active_monthly_1,active_monthly_2,active_monthly_3,...,valley_good_days_to_debit_trans_history_ratio250,valley_good_days_to_debit_trans_history_ratio500,valley_most_recent_gtd_length_100,valley_most_recent_gtd_length_250,valley_most_recent_gtd_length_500,valley_trans_history_ratio_100,valley_trans_history_ratio_250,valley_trans_history_ratio_500,currentBalance,IBVStatusID
0,931d1df2-04c6-41a8-a2f2-190598c10c69,0.00,1,0,0,0,47.73,0,0,0.00,...,1.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,86334
1,e7498950-e42b-4b6b-81d6-e09362e5c05c,4987.71,2,0,0,1,18172.68,0,0,4987.71,...,0.645161,0.623656,16.0,16.0,18.0,0.064516,0.032258,0.021505,4.24,86334
2,ea265c15-947a-4377-875d-829ae3d39f89,0.00,1,0,0,0,3775.66,0,0,0.00,...,1.000000,1.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.00,86334


In [18]:
# --- Join features to FPDAA labels (keep only labeled rows) ---
label_cols = ["IBVStatusID", "FPDAA", "FPDAA_matured"]

train_df = features_df.merge(
    df_perf_orig[label_cols],
    on="IBVStatusID",
    how="inner",
)

print("features rows:", len(features_df))
print("labeled rows (df_perf_orig):", len(df_perf_orig))
print("joined rows:", len(train_df))

train_df.head()


features rows: 11843
labeled rows (df_perf_orig): 5704
joined rows: 5951


,accountGuid,activeMonthlyIncome,active_count_0,active_count_1,active_count_2,active_count_3,active_monthly_0,active_monthly_1,active_monthly_2,active_monthly_3,...,valley_most_recent_gtd_length_100,valley_most_recent_gtd_length_250,valley_most_recent_gtd_length_500,valley_trans_history_ratio_100,valley_trans_history_ratio_250,valley_trans_history_ratio_500,currentBalance,IBVStatusID,FPDAA,FPDAA_matured
0,ceb777fe-2bfd-4e3b-b0fd-ccf4cc7ab6c9,5287.21,1,0,0,1,4984.64,0.00,0.0,5287.21,...,5.0,5.0,5.0,0.064516,0.053763,0.032258,0.63,78484,0,0.0
1,83747970-5f2e-4fcb-a06b-fac4eab1650d,4558.19,1,0,0,1,3260.80,0.00,0.0,4558.19,...,9.0,9.0,9.0,0.086022,0.053763,0.043011,-261.97,63698,0,0.0
2,83747970-5f2e-4fcb-a06b-fac4eab1650d,4558.19,1,0,0,1,3260.80,0.00,0.0,4558.19,...,9.0,9.0,9.0,0.086022,0.053763,0.043011,-261.97,63698,0,0.0
3,752e6600-1faa-4918-aa1f-ae9372ae7c0c,3807.85,3,1,0,2,9491.05,346.46,0.0,3807.85,...,1.0,1.0,4.0,0.096774,0.075269,0.032258,-749.64,53384,0,0.0
4,752e6600-1faa-4918-aa1f-ae9372ae7c0c,3807.85,3,1,0,2,9491.05,346.46,0.0,3807.85,...,1.0,1.0,4.0,0.096774,0.075269,0.032258,-749.64,53384,0,0.0


In [19]:
train_df.shape

(5951, 106)

In [20]:
# --- Export unseen feature rows (non-labeled IBVStatusID) ---
output_dir = "/Users/starsrain/jan2026_concord/jan2026_loonie_customize/retrain_data/unseen_testing_data"
output_path = f"{output_dir}/unseen_features.parquet"

# Ensure we exclude labeled IBVStatusID rows used in train_df
labeled_ids = set(train_df["IBVStatusID"].dropna().unique())

unseen_df = features_df[~features_df["IBVStatusID"].isin(labeled_ids)].copy()

# Persist unseen features
from pathlib import Path
Path(output_dir).mkdir(parents=True, exist_ok=True)

unseen_df.to_parquet(output_path, index=False)
print("unseen rows:", len(unseen_df))
print("saved:", output_path)


unseen rows: 8166
saved: /Users/starsrain/jan2026_concord/jan2026_loonie_customize/retrain_data/unseen_testing_data/unseen_features.parquet


In [ ]:
# --- Export labeled dataset for AUC notebook ---
""" output_path = "/Users/starsrain/jan2026_concord/jan2026_loonie_customize/retrain_data/features_with_fpdaa.parquet"
train_df.to_parquet(output_path, index=False)
print("saved:", output_path) """


saved: /Users/starsrain/jan2026_concord/jan2026_loonie_customize/retrain_data/features_with_fpdaa.parquet
